<a href="https://colab.research.google.com/github/bcalm-2/gen-ai-assignment-1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install torch torchvision matplotlib reportlab

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os

from torch.utils.data import DataLoader

In [29]:
transform = transforms.ToTensor()

dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

loader = DataLoader(dataset, batch_size=128, shuffle=True)

print("Dataset Loaded")

Dataset Loaded


In [30]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        return self.fc21(h), self.fc22(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE().to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

for epoch in range(3):
    total_loss = 0
    for data, _ in loader:
        data = data.view(-1, 784).to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss}")

Epoch 1, Loss: 9923331.002929688
Epoch 2, Loss: 7300006.7138671875
Epoch 3, Loss: 6871668.201171875


In [32]:
vae.eval()

with torch.no_grad():
    z = torch.randn(16, 20).to(device)
    sample = vae.decode(z).cpu()

fig = plt.figure(figsize=(4,4))

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(sample[i].view(28,28), cmap='gray')
    plt.axis('off')

plt.tight_layout()

fig.savefig("vae_output.png", dpi=300)
plt.close(fig)

print("Image Saved Successfully")

Image Saved Successfully


In [33]:
print(os.listdir())

['.config', 'data', 'Final_Lab_Report.pdf', 'Generative_AI_Lab_Assignment_1.pdf', 'vae_output.png', 'Generative_AI_Lab_Assignment_Final.pdf', 'Final_Generative_AI_Lab_Assignment.pdf', 'Generative_AI_Lab_Assignment_1_Professional.pdf', 'sample_data']


In [34]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

doc = SimpleDocTemplate("Generative_AI_Lab_Assignment_Final.pdf")
elements = []
styles = getSampleStyleSheet()

# Title Page
elements.append(Paragraph("<b>GENERATIVE AI LAB ASSIGNMENT – 1</b>", styles["Title"]))
elements.append(Spacer(1, 0.5 * inch))

elements.append(Paragraph("Name: Srashti Dwivedi", styles["Normal"]))
elements.append(Paragraph("Programme: MCA", styles["Normal"]))
elements.append(Paragraph("Subject: Generative AI Lab", styles["Normal"]))
elements.append(Spacer(1, 0.5 * inch))

elements.append(PageBreak())

# Objective
elements.append(Paragraph("<b>Objective</b>", styles["Heading2"]))
elements.append(Spacer(1, 0.2 * inch))

elements.append(Paragraph(
    "To implement Variational Autoencoder (VAE) to generate handwritten digit images "
    "using MNIST dataset and analyze generative modeling behavior.",
    styles["Normal"]
))
elements.append(Spacer(1, 0.5 * inch))

# Output Section
elements.append(Paragraph("<b>VAE Generated Output</b>", styles["Heading2"]))
elements.append(Spacer(1, 0.3 * inch))

img = Image("vae_output.png", width=4*inch, height=4*inch)
elements.append(img)

elements.append(Spacer(1, 0.5 * inch))

# Conclusion
elements.append(Paragraph("<b>Conclusion</b>", styles["Heading2"]))
elements.append(Spacer(1, 0.2 * inch))

elements.append(Paragraph(
    "The VAE model successfully generated synthetic handwritten digits. "
    "Generated images appear smooth but slightly blurred due to reconstruction-based training.",
    styles["Normal"]
))

doc.build(elements)

print("FINAL PROFESSIONAL PDF GENERATED SUCCESSFULLY!")

FINAL PROFESSIONAL PDF GENERATED SUCCESSFULLY!


In [35]:
from google.colab import files
files.download("Generative_AI_Lab_Assignment_Final.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>